# load data

In [1]:
from glob import glob
import os
images=[]
labels=[]
for folder in os.listdir('../dataset/wrist_xray'):
    for pic in glob('../dataset/wrist_xray/{}/*.jpg'.format(folder)):
        images.append(pic)
        labels.append(folder)
        

In [2]:
import pandas as pd
data=pd.DataFrame(zip(images,labels),columns=['img','label'])
data.label=data.label.map({'Fracture':0,'Normal':1})
data = data.sample(frac=1).reset_index(drop=True)
data.head()

,img,label
0,../dataset/wrist_xray/Fracture\89 Male (L View...,0
1,../dataset/wrist_xray/Fracture\48 Male (A View...,0
2,../dataset/wrist_xray/Fracture\1 Male (L View)...,0
3,../dataset/wrist_xray/Fracture\88 Male (A View...,0
4,../dataset/wrist_xray/Fracture\34 Male (A View...,0


# top 5 lightest models in torchvision
1	squeezenet1_1	1235496  
2	shufflenet_v2_x0_5	1366792  
3	mnasnet0_5	2218512  
4	mobilenet_v2	3504872  
5	densenet121	7978856  `

In [3]:
import sys
sys.path.append("../src") 

In [4]:
from models.squeezenet import squeezenet1_1
from models.mymodels import mymodel4
from models.shufflenet import shufflenet_v2_x0_5
from models.mnastnet import mnasnet0_5
from models.densenet import densenet121
from models.mobilenet import mobilenet
from cross_vals import kfoldcv

In [5]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)/1e6

In [6]:
print('MyNet parameters :' ,count_parameters(mymodel4()))
print('SqueezeNet parameters :', count_parameters(squeezenet1_1()))
print('ShuffleNet parameters :', count_parameters(shufflenet_v2_x0_5()))
print('MnastNet parameters :', count_parameters(mnasnet0_5()))
print('MobileNet parameters :', count_parameters(mobilenet()))
print('DenseNet parameters :', count_parameters(densenet121()))


MyNet parameters : 0.473328
SqueezeNet parameters : 0.787008
ShuffleNet parameters : 0.47256
MnastNet parameters : 1.101192
MobileNet parameters : 2.387264
DenseNet parameters : 7.472384


In [7]:
epoch=32
batchsize=8
lr=0.001

dim='3D' or '2D' for 3D images and 2D iamges
skip_tuning=False, no grid search applied for hyper-parameter tuning, 
aug=1-> pytorch augmentation
aug=0-> no augmentation
aug=2-> albumentation augmentation
aug=3-> rand augmentation

# MyNet

In [8]:
%%time
train_cv,clf_report=kfoldcv(model=mymodel4(),epochs=epoch,batchsize=batchsize,data=data,\
                            lr=0.001,skip_tuning=False,aug=1,dim='2D',pca=False)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.916   0.872     0.886
Class 1       0.860   0.874     0.856
macro         0.886   0.872     0.870
weighted      0.890   0.874     0.874
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.932   0.836     0.876
Class 1       0.816   0.914     0.858
macro         0.874   0.872     0.868
weighted      0.882   0.870     0.868
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.946   0.826     0.878
Class 1       0.808   0.940     0.866
macro         0.876   0.880     0.874
weighted      0.886   0.876     0.874
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.936   0.870     0.898
Class 1       0.854   0.914     0.878
macro         0.894   0.894     0.890
weighted      0.898   0.892     0.890
--------------------------Accuracy Tab

,DT,SVM,LR,KNN
0,0.87,0.87,0.87,0.85
1,0.82,0.82,0.82,0.85
2,0.87,0.85,0.85,0.87
3,0.89,0.89,0.92,0.97
4,0.92,0.92,0.92,0.92
mean,0.874,0.87,0.876,0.892


In [9]:
%%time
train_cv,clf_report=kfoldcv(model=mymodel4(),epochs=epoch,batchsize=batchsize,data=data,lr=0.001,skip_tuning=False,aug=0)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.620   0.558     0.584
Class 1       0.472   0.534     0.498
macro         0.544   0.544     0.540
weighted      0.560   0.550     0.550
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.776   0.780     0.778
Class 1       0.702   0.692     0.692
macro         0.742   0.736     0.734
weighted      0.748   0.746     0.744
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.734   0.756     0.742
Class 1       0.662   0.630     0.644
macro         0.698   0.692     0.694
weighted      0.708   0.706     0.704
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.702   0.658     0.674
Class 1       0.560   0.598     0.566
macro         0.630   0.630     0.622
weighted      0.648   0.632     0.630
--------------------------Accuracy Tab

,DT,SVM,LR,KNN
0,0.51,0.82,0.77,0.64
1,0.56,0.67,0.67,0.64
2,0.54,0.74,0.69,0.67
3,0.61,0.71,0.74,0.55
4,0.53,0.79,0.66,0.66
mean,0.55,0.746,0.706,0.632


# SqueezeNet

In [10]:
%%time
train_cv,clf_report=kfoldcv(model=squeezenet1_1(),epochs=epoch,batchsize=batchsize,data=data,lr=0.001,skip_tuning=False,aug=1)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.642   0.656     0.644
Class 1       0.518   0.502     0.506
macro         0.580   0.580     0.576
weighted      0.592   0.592     0.588
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.658   0.690     0.668
Class 1       0.558   0.516     0.526
macro         0.610   0.606     0.596
weighted      0.622   0.610     0.608
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.660   0.716     0.684
Class 1       0.570   0.502     0.526
macro         0.618   0.610     0.606
weighted      0.630   0.620     0.620
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.648   0.640     0.642
Class 1       0.468   0.486     0.474
macro         0.558   0.564     0.560
weighted      0.580   0.582     0.578
--------------------------Accuracy Tab

,DT,SVM,LR,KNN
0,0.95,0.92,0.95,0.85
1,0.54,0.56,0.59,0.46
2,0.49,0.62,0.62,0.62
3,0.45,0.42,0.39,0.61
4,0.53,0.53,0.55,0.37
mean,0.592,0.61,0.62,0.582


In [11]:
%%time
train_cv,clf_report=kfoldcv(model=squeezenet1_1(),epochs=epoch,batchsize=batchsize,data=data,lr=0.001,skip_tuning=False,aug=0)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.578   0.536     0.546
Class 1       0.406   0.448     0.414
macro         0.492   0.492     0.482
weighted      0.510   0.492     0.488
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.572   0.634     0.590
Class 1       0.450   0.358     0.360
macro         0.512   0.498     0.474
weighted      0.524   0.508     0.488
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.582   0.552     0.556
Class 1       0.400   0.442     0.410
macro         0.492   0.498     0.484
weighted      0.508   0.498     0.492
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.546   0.650     0.592
Class 1       0.402   0.286     0.324
macro         0.474   0.470     0.458
weighted      0.490   0.492     0.478
--------------------------Accuracy Tab

,DT,SVM,LR,KNN
0,0.56,0.59,0.62,0.62
1,0.46,0.62,0.46,0.62
2,0.44,0.38,0.38,0.38
3,0.58,0.45,0.58,0.47
4,0.42,0.50,0.45,0.37
mean,0.492,0.508,0.498,0.492


# ShuffleNet

In [12]:
%%time
train_cv,clf_report=kfoldcv(model=shufflenet_v2_x0_5(),epochs=epoch,batchsize=batchsize,data=data,lr=0.001,skip_tuning=False,aug=1)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.648   0.652     0.644
Class 1       0.542   0.516     0.516
macro         0.594   0.584     0.580
weighted      0.606   0.602     0.596
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.670   0.638     0.652
Class 1       0.540   0.574     0.554
macro         0.602   0.606     0.604
weighted      0.620   0.610     0.614
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.652   0.648     0.650
Class 1       0.530   0.536     0.532
macro         0.592   0.594     0.590
weighted      0.604   0.602     0.602
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.722   0.716     0.706
Class 1       0.594   0.574     0.554
macro         0.656   0.646     0.632
weighted      0.670   0.664     0.644
--------------------------Accuracy Tab

,DT,SVM,LR,KNN
0,0.46,0.64,0.59,0.56
1,0.54,0.56,0.49,0.67
2,0.62,0.59,0.64,0.69
3,0.71,0.63,0.61,0.74
4,0.68,0.63,0.68,0.66
mean,0.602,0.61,0.602,0.664


In [13]:
%%time
train_cv,clf_report=kfoldcv(model=shufflenet_v2_x0_5(),epochs=epoch,batchsize=batchsize,data=data,lr=0.001,skip_tuning=False,aug=0)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.612   0.598     0.594
Class 1       0.462   0.484     0.464
macro         0.536   0.540     0.526
weighted      0.550   0.542     0.532
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.592   0.680     0.630
Class 1       0.440   0.356     0.386
macro         0.516   0.518     0.506
weighted      0.534   0.540     0.526
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.610   0.610     0.606
Class 1       0.456   0.462     0.456
macro         0.534   0.534     0.530
weighted      0.550   0.544     0.546
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.630   0.668     0.646
Class 1       0.504   0.462     0.482
macro         0.566   0.566     0.564
weighted      0.578   0.586     0.580
--------------------------Accuracy Tab

,DT,SVM,LR,KNN
0,0.62,0.54,0.56,0.49
1,0.44,0.41,0.36,0.46
2,0.44,0.62,0.69,0.56
3,0.71,0.63,0.61,0.68
4,0.50,0.50,0.50,0.74
mean,0.542,0.54,0.544,0.586


# Mnasnet

In [14]:
%%time
train_cv,clf_report=kfoldcv(model=mnasnet0_5(),epochs=epoch,batchsize=batchsize,data=data,lr=0.001,skip_tuning=False,aug=1)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.832   0.786     0.806
Class 1       0.704   0.756     0.730
macro         0.770   0.772     0.770
weighted      0.780   0.778     0.778
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.886   0.804     0.840
Class 1       0.734   0.826     0.766
macro         0.806   0.814     0.804
weighted      0.822   0.814     0.812
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.864   0.804     0.830
Class 1       0.746   0.814     0.776
macro         0.802   0.808     0.802
weighted      0.812   0.808     0.806
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.844   0.778     0.806
Class 1       0.720   0.788     0.748
macro         0.780   0.782     0.776
weighted      0.792   0.782     0.782
--------------------------Accuracy Tab

,DT,SVM,LR,KNN
0,0.46,0.62,0.62,0.51
1,0.90,0.87,0.87,0.87
2,0.90,0.90,0.87,0.87
3,0.89,0.89,0.89,0.84
4,0.74,0.79,0.79,0.82
mean,0.778,0.814,0.808,0.782


In [15]:
%%time
train_cv,clf_report=kfoldcv(model=mnasnet0_5(),epochs=epoch,batchsize=batchsize,data=data,lr=0.001,skip_tuning=False,aug=0)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.602   0.610     0.596
Class 1       0.490   0.470     0.460
macro         0.548   0.542     0.530
weighted      0.562   0.542     0.540
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.616   0.690     0.644
Class 1       0.498   0.404     0.426
macro         0.556   0.550     0.536
weighted      0.570   0.576     0.558
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.630   0.674     0.648
Class 1       0.496   0.454     0.468
macro         0.562   0.564     0.556
weighted      0.576   0.582     0.572
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.606   0.618     0.604
Class 1       0.480   0.460     0.456
macro         0.544   0.538     0.530
weighted      0.560   0.544     0.542
--------------------------Accuracy Tab

,DT,SVM,LR,KNN
0,0.62,0.46,0.56,0.38
1,0.56,0.54,0.44,0.62
2,0.56,0.62,0.62,0.56
3,0.42,0.63,0.66,0.53
4,0.55,0.63,0.63,0.63
mean,0.542,0.576,0.582,0.544


# MobileNet

In [16]:
%%time
train_cv,clf_report=kfoldcv(model=mobilenet(),epochs=epoch,batchsize=batchsize,data=data,lr=0.001,skip_tuning=False,aug=1)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.882   0.810     0.840
Class 1       0.778   0.852     0.810
macro         0.832   0.834     0.826
weighted      0.838   0.828     0.828
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.942   0.828     0.878
Class 1       0.810   0.926     0.858
macro         0.878   0.880     0.870
weighted      0.886   0.872     0.872
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.908   0.838     0.868
Class 1       0.812   0.870     0.832
macro         0.864   0.856     0.850
weighted      0.868   0.854     0.854
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.910   0.822     0.860
Class 1       0.792   0.888     0.832
macro         0.854   0.854     0.848
weighted      0.862   0.850     0.852
--------------------------Accuracy Tab

,DT,SVM,LR,KNN
0,0.79,0.77,0.77,0.74
1,0.72,0.87,0.87,0.87
2,0.90,0.90,0.92,0.90
3,0.89,0.95,0.89,0.92
4,0.84,0.87,0.82,0.82
mean,0.828,0.872,0.854,0.85


In [17]:
%%time
train_cv,clf_report=kfoldcv(model=mobilenet(),epochs=epoch,batchsize=batchsize,data=data,lr=0.001,skip_tuning=False,aug=0)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.640   0.626     0.634
Class 1       0.504   0.520     0.512
macro         0.570   0.572     0.568
weighted      0.586   0.586     0.586
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.604   0.610     0.600
Class 1       0.532   0.510     0.510
macro         0.566   0.558     0.554
weighted      0.576   0.570     0.564
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.664   0.632     0.638
Class 1       0.560   0.572     0.548
macro         0.614   0.604     0.592
weighted      0.626   0.602     0.598
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.604   0.710     0.652
Class 1       0.510   0.376     0.426
macro         0.560   0.542     0.538
weighted      0.570   0.570     0.558
--------------------------Accuracy Tab

,DT,SVM,LR,KNN
0,0.49,0.36,0.54,0.56
1,0.77,0.74,0.74,0.67
2,0.51,0.51,0.54,0.51
3,0.66,0.61,0.53,0.53
4,0.50,0.63,0.66,0.58
mean,0.586,0.57,0.602,0.57


# DenseNet121

In [18]:
%%time
train_cv,clf_report=kfoldcv(model=densenet121(),epochs=epoch,batchsize=batchsize,data=data,lr=0.001,skip_tuning=False,aug=1)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.814   0.828     0.820
Class 1       0.768   0.748     0.754
macro         0.792   0.790     0.786
weighted      0.800   0.790     0.790
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.872   0.856     0.860
Class 1       0.824   0.840     0.826
macro         0.848   0.852     0.842
weighted      0.858   0.842     0.844
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.890   0.828     0.852
Class 1       0.802   0.864     0.824
macro         0.846   0.848     0.838
weighted      0.856   0.838     0.840
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.854   0.802     0.820
Class 1       0.756   0.814     0.780
macro         0.804   0.808     0.800
weighted      0.818   0.800     0.804
--------------------------Accuracy Tab

,DT,SVM,LR,KNN
0,0.92,0.92,0.92,0.87
1,0.72,0.77,0.79,0.79
2,0.79,0.79,0.77,0.74
3,0.68,0.84,0.84,0.71
4,0.84,0.89,0.87,0.89
mean,0.79,0.842,0.838,0.8


In [19]:
%%time
train_cv,clf_report=kfoldcv(model=densenet121(),epochs=epoch,batchsize=batchsize,data=data,lr=0.001,skip_tuning=False,aug=0)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.650   0.648     0.646
Class 1       0.512   0.518     0.512
macro         0.580   0.584     0.578
weighted      0.596   0.596     0.594
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.762   0.796     0.780
Class 1       0.708   0.668     0.684
macro         0.736   0.728     0.732
weighted      0.742   0.742     0.740
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.778   0.806     0.788
Class 1       0.716   0.680     0.690
macro         0.748   0.742     0.740
weighted      0.756   0.752     0.750
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.642   0.648     0.640
Class 1       0.510   0.500     0.500
macro         0.578   0.574     0.570
weighted      0.590   0.592     0.586
--------------------------Accuracy Tab

,DT,SVM,LR,KNN
0,0.62,0.72,0.72,0.67
1,0.72,0.67,0.77,0.59
2,0.54,0.82,0.85,0.62
3,0.55,0.68,0.68,0.50
4,0.55,0.82,0.74,0.58
mean,0.596,0.742,0.752,0.592
